In [154]:
import numpy as np
import pandas as pd
import math
import csv

In [ ]:
"""
Assume that L1 Instruction Cache access time is 0, since it is perfect.

cpi for each cache structure

gmean for everything except cycle time
cpu_time = instruction_per_program * cpi * cycle_time


cpi = 1.2 + ((# of d miss)/(# of instructions)) * math.ceil(miss_penalty/cycle time)
where
# of d miss: from benchmark
# of instruction: given
# miss_penalty: 100 + 5 * (line_size - 1) 
# cycle_time: given

geometric mean of each benchmark cpi will be used to calculate the cpu_time because we have different cycle time
"""

In [128]:
s: dict = {
    "bfs_matrix": [11201166 ,15822, 139465 + 132512, 64957 + 251, 0.15583],
    "bfs_pointer": [320968, 2208, 3113+2813, 803 + 3, 0.06782],
    "conv2D": [5108809, 3234, 12090 + 7823, 8288 + 1458, 0.04667],
    "fir": [389940121, 8325, 81174 + 65914, 17715 + 2218, 0.08939],
    "kmean": [625854212, 4278, 210645 + 20091, 3734 + 91, 0.01479],
    "mm": [43389464, 3179, 794712 + 10274, 2300 + 7, 0.00282],
    "quicksort": [17430826, 374052, 114683 + 64995, 6203 + 112, 0.00913],
    "sobel": [48564237, 4395, 28342 + 33688, 3423 + 25, 0.003411]
}
number_of_ins = [11201166, 320968, 5108809, 389940121, 625854212, 43389464, 17430826, 48564237]

In [127]:
# gmean custom function
def g_mean(x):
    a = np.log(x)
    return np.exp(a.mean())

In [143]:
instruction_per_program: list = [i[0] for i in list(s.values())]
instruction_per_program_gmean = np.array(g_mean(a))

In [107]:
cache_size = np.array([pow(2, 12), pow(2, 13), pow(2, 14), pow(2, 15), pow(2, 16), pow(2, 17), pow(2, 18), pow(2, 19), pow(2, 20), pow(2, 21)])
cache_index = np.log2(cache_size) - 11

cache_index_set: set = {}
# print(cache_index.shape[0]) -> array size for indexing
for i in range(cache_index.shape[0]):
    cache_index_set[cache_size[i]] = int(cache_index[i])

print(cache_index_set)

{4096: 1, 8192: 2, 16384: 3, 32768: 4, 65536: 5, 131072: 6, 262144: 7, 524288: 8, 1048576: 9, 2097152: 10}


In [96]:
associative = np.array([1, 2, 4, 8, 16, 32, 64])

associative_index_set: set = {}
for i in range(associative.shape[0]):
    associative_index_set[associative[i]] = int(np.log2(associative[i]))

associative_index_set

{1: 0, 2: 1, 4: 2, 8: 3, 16: 4, 32: 5, 64: 6}

In [68]:
block_size = np.array([16, 32])
line_size = block_size//4

In [57]:
base_command = f"--dc="

In [34]:
print(associative_index[0])
associative[int(associative_index[0])]

0.0


1

In [55]:
cache_access_4 = pd.read_csv("4_words.csv").to_numpy()
print(cache_access_4)

[[ 1.    0.21  0.27  0.32  0.38  0.46  0.54  0.69  0.85  1.14  1.35]
 [ 2.    0.37  0.44  0.5   0.54  0.57  0.67  0.73  1.    1.24  1.51]
 [ 4.    0.44  0.49  0.52  0.7   0.61  0.69  0.79  1.02  1.31  1.58]
 [ 8.     nan  0.67  0.7   0.75  0.77  0.83  0.9   1.18  1.61  1.98]
 [16.     nan   nan  1.05  1.08  1.13  1.17  1.24  1.46  2.43  2.74]
 [32.     nan   nan   nan  1.83  1.84  1.91  2.    2.44  2.85  4.97]
 [64.     nan   nan   nan   nan  3.39  3.67  3.87  4.25  4.72  6.06]]


In [56]:
cache_access_8 = pd.read_csv("8_words.csv").to_numpy()
print(cache_access_8)
# print(cache_access_8[0][1])

[[ 1.    0.24  0.28  0.3   0.36  0.47  0.54  0.7   0.92  1.15  1.62]
 [ 2.    0.51  0.53  0.54  0.56  0.6   0.65  0.75  0.96  1.27  1.8 ]
 [ 4.     nan  0.73  0.74  0.76  0.8   0.83  0.87  1.07  1.44  2.21]
 [ 8.     nan   nan  1.13  1.14  1.18  1.21  1.24  1.66  2.24  3.14]
 [16.     nan   nan   nan  1.8   1.94  1.97  1.98  2.63  3.34  4.47]
 [32.     nan   nan   nan   nan  2.55  2.37  3.56  3.85  4.65  6.35]
 [64.     nan   nan   nan   nan   nan  4.5   5.23  5.78  6.47  7.2 ]]


In [ ]:
# equation
# cpi = 1.2 + ((# of d miss)/(# of instructions)) * math.ceil(miss_penalty/cycle time)


In [60]:
bm_pd = pd.read_csv("output.csv")

In [65]:
bm = bm_pd.values.tolist()

for b in bm:
    cache_data = b[0][5:]
    cache_data_list = cache_data.split(":")
    data_cache_entry_num = int(cache_data_list[0])
    data_cache_associativity = int(cache_data_list[1])
    data_cache_line_size = int(cache_data_list[2])//4

    data_cache_info = list((data_cache_entry_num, data_cache_associativity, data_cache_line_size))
    b.extend(data_cache_info)
    
    program_name: str = b[1].split("/")[-1]
    b[1] = program_name
    
    data_cache_miss_num: int = b[2]

    b.pop(0)



In [67]:
bm.append(['bfs_matrix', 543834, 256, 1, 4])

bm = sorted(bm, key=lambda b: b[2])
bm = sorted(bm, key=lambda b: b[3])
bm = sorted(bm, key=lambda b: b[4])
# bm
# bm = sorted(sorted(sorted(bm, key = lambda b: b[2]), key = lambda b: b[3]), key = lambda b: b[4])
bm

# print(bm)
# 7 programs
# 0 ~ 6
# 7 ~ 13
# ...

[['bfs_pointer', 9941, 256, 1, 4],
 ['conv2D', 30513, 256, 1, 4],
 ['fir', 3926638, 256, 1, 4],
 ['kmean', 2478740, 256, 1, 4],
 ['mm', 447167, 256, 1, 4],
 ['quicksort', 3352240, 256, 1, 4],
 ['sobel', 576650, 256, 1, 4],
 ['bfs_matrix', 543834, 256, 1, 4],
 ['bfs_matrix', 527756, 512, 1, 4],
 ['bfs_pointer', 8894, 512, 1, 4],
 ['conv2D', 24628, 512, 1, 4],
 ['fir', 3864093, 512, 1, 4],
 ['kmean', 2460933, 512, 1, 4],
 ['mm', 428564, 512, 1, 4],
 ['quicksort', 3302713, 512, 1, 4],
 ['sobel', 570041, 512, 1, 4],
 ['bfs_matrix', 518257, 1024, 1, 4],
 ['bfs_pointer', 7683, 1024, 1, 4],
 ['conv2D', 18529, 1024, 1, 4],
 ['fir', 3828015, 1024, 1, 4],
 ['kmean', 2445561, 1024, 1, 4],
 ['mm', 348747, 1024, 1, 4],
 ['quicksort', 3263097, 1024, 1, 4],
 ['sobel', 567259, 1024, 1, 4],
 ['bfs_matrix', 513180, 2048, 1, 4],
 ['bfs_pointer', 6968, 2048, 1, 4],
 ['conv2D', 15603, 2048, 1, 4],
 ['fir', 3811240, 2048, 1, 4],
 ['kmean', 2440874, 2048, 1, 4],
 ['mm', 307232, 2048, 1, 4],
 ['quicksort', 31

In [138]:
specification: list[list[int]] = []
cache_miss_per_diff_cache: list[list[int]] = []

a = []

for i in range(len(bm)):
    if not (i % 8): # if i is divisible by 8
        # cache miss number -> list
        if a:
            # print(a)
            cache_miss_per_diff_cache.append(a)
            a = []

        # data cycle
        
        
        # cache construction data
        specification.append(bm[i][2:])
    a.append(bm[i][1])

cache_miss_per_diff_cache.append(a)
tmp = cache_miss_per_diff_cache[0].pop(-1)
cache_miss_per_diff_cache[0].insert(0, tmp)
# specification
cache_miss_per_diff_cache

# print(len(specification), len(cache_miss_per_diff_cache))

[[543834, 9941, 30513, 3926638, 2478740, 447167, 3352240, 576650],
 [527756, 8894, 24628, 3864093, 2460933, 428564, 3302713, 570041],
 [518257, 7683, 18529, 3828015, 2445561, 348747, 3263097, 567259],
 [513180, 6968, 15603, 3811240, 2440874, 307232, 3191233, 565957],
 [511395, 5097, 13449, 3802877, 2430627, 135262, 3161384, 564992],
 [510279, 5033, 10249, 3798687, 2093611, 24665, 3141962, 540571],
 [508894, 5033, 10154, 3780282, 2085763, 15036, 3132880, 538604],
 [504275, 5033, 10154, 3779851, 2084981, 15036, 3131905, 538504],
 [489317, 5033, 10154, 3763915, 2084981, 15036, 3131662, 538504],
 [438198, 5033, 10154, 3763798, 2084981, 15036, 3131662, 538504],
 [534169, 10030, 21621, 3826921, 2454807, 343809, 3291018, 560066],
 [528610, 8858, 17629, 3805300, 2446771, 299977, 3251174, 553917],
 [517792, 7762, 14486, 3801541, 2443589, 292657, 3218059, 551427],
 [512001, 6738, 13591, 3797898, 2435654, 264055, 3190366, 550682],
 [509673, 5898, 12602, 3796959, 2385362, 124735, 3164115, 549573],

In [137]:
specification

[[256, 1, 4],
 [512, 1, 4],
 [1024, 1, 4],
 [2048, 1, 4],
 [4096, 1, 4],
 [8192, 1, 4],
 [16384, 1, 4],
 [32768, 1, 4],
 [65536, 1, 4],
 [131072, 1, 4],
 [128, 2, 4],
 [256, 2, 4],
 [512, 2, 4],
 [1024, 2, 4],
 [2048, 2, 4],
 [4096, 2, 4],
 [8192, 2, 4],
 [16384, 2, 4],
 [32768, 2, 4],
 [65536, 2, 4],
 [64, 4, 4],
 [128, 4, 4],
 [256, 4, 4],
 [512, 4, 4],
 [1024, 4, 4],
 [2048, 4, 4],
 [4096, 4, 4],
 [8192, 4, 4],
 [16384, 4, 4],
 [32768, 4, 4],
 [32, 8, 4],
 [64, 8, 4],
 [128, 8, 4],
 [256, 8, 4],
 [512, 8, 4],
 [1024, 8, 4],
 [2048, 8, 4],
 [4096, 8, 4],
 [8192, 8, 4],
 [16384, 8, 4],
 [16, 16, 4],
 [32, 16, 4],
 [64, 16, 4],
 [128, 16, 4],
 [256, 16, 4],
 [512, 16, 4],
 [1024, 16, 4],
 [2048, 16, 4],
 [4096, 16, 4],
 [8192, 16, 4],
 [8, 32, 4],
 [16, 32, 4],
 [32, 32, 4],
 [64, 32, 4],
 [128, 32, 4],
 [256, 32, 4],
 [512, 32, 4],
 [1024, 32, 4],
 [2048, 32, 4],
 [4096, 32, 4],
 [4, 64, 4],
 [8, 64, 4],
 [16, 64, 4],
 [32, 64, 4],
 [64, 64, 4],
 [128, 64, 4],
 [256, 64, 4],
 [512, 64

In [113]:
# cycle time list
cycle_time_list: list[float] = []
for i in specification:
    size_indx =  cache_index_set[i[0] * i[1] * i[2] * 4]
    asso_indx = associative_index_set[i[1]]
    if i[2] == 4: # go to 4
        cycle_time = cache_access_4[asso_indx][size_indx]
    else: # go to 8
        cycle_time = cache_access_8[asso_indx][size_indx]

    # if not cycle_time:
    #     cycle_time_list.append(None)
    # else:
    cycle_time_list.append(max(0.6, cycle_time))

print(cycle_time_list)

[0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.69, 0.85, 1.14, 1.35, 0.6, 0.6, 0.6, 0.6, 0.6, 0.67, 0.73, 1.0, 1.24, 1.51, 0.6, 0.6, 0.6, 0.7, 0.61, 0.69, 0.79, 1.02, 1.31, 1.58, 0.6, 0.67, 0.7, 0.75, 0.77, 0.83, 0.9, 1.18, 1.61, 1.98, 0.6, 0.6, 1.05, 1.08, 1.13, 1.17, 1.24, 1.46, 2.43, 2.74, 0.6, 0.6, 0.6, 1.83, 1.84, 1.91, 2.0, 2.44, 2.85, 4.97, 0.6, 0.6, 0.6, 0.6, 3.39, 3.67, 3.87, 4.25, 4.72, 6.06, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.7, 0.92, 1.15, 1.62, 0.6, 0.6, 0.6, 0.6, 0.6, 0.65, 0.75, 0.96, 1.27, 1.8, 0.6, 0.73, 0.74, 0.76, 0.8, 0.83, 0.87, 1.07, 1.44, 2.21, 0.6, 0.6, 1.13, 1.14, 1.18, 1.21, 1.24, 1.66, 2.24, 3.14, 0.6, 0.6, 0.6, 1.8, 1.94, 1.97, 1.98, 2.63, 3.34, 4.47, 0.6, 0.6, 0.6, 0.6, 2.55, 2.37, 3.56, 3.85, 4.65, 6.35, 0.6, 0.6, 0.6, 0.6, 0.6, 4.5, 5.23, 5.78, 6.47, 7.2]


In [116]:
# mp: int = 100 + 0.5 * (asso - 1)
miss_cost_list = []

for i in specification:
    if i[2] == 4:
        miss_cost_list.append(101.5)
    else:
        miss_cost_list.append(103.5)

[256, 1, 4]: 101.5
[512, 1, 4]: 101.5
[1024, 1, 4]: 101.5
[2048, 1, 4]: 101.5
[4096, 1, 4]: 101.5
[8192, 1, 4]: 101.5
[16384, 1, 4]: 101.5
[32768, 1, 4]: 101.5
[65536, 1, 4]: 101.5
[131072, 1, 4]: 101.5
[128, 2, 4]: 101.5
[256, 2, 4]: 101.5
[512, 2, 4]: 101.5
[1024, 2, 4]: 101.5
[2048, 2, 4]: 101.5
[4096, 2, 4]: 101.5
[8192, 2, 4]: 101.5
[16384, 2, 4]: 101.5
[32768, 2, 4]: 101.5
[65536, 2, 4]: 101.5
[64, 4, 4]: 101.5
[128, 4, 4]: 101.5
[256, 4, 4]: 101.5
[512, 4, 4]: 101.5
[1024, 4, 4]: 101.5
[2048, 4, 4]: 101.5
[4096, 4, 4]: 101.5
[8192, 4, 4]: 101.5
[16384, 4, 4]: 101.5
[32768, 4, 4]: 101.5
[32, 8, 4]: 101.5
[64, 8, 4]: 101.5
[128, 8, 4]: 101.5
[256, 8, 4]: 101.5
[512, 8, 4]: 101.5
[1024, 8, 4]: 101.5
[2048, 8, 4]: 101.5
[4096, 8, 4]: 101.5
[8192, 8, 4]: 101.5
[16384, 8, 4]: 101.5
[16, 16, 4]: 101.5
[32, 16, 4]: 101.5
[64, 16, 4]: 101.5
[128, 16, 4]: 101.5
[256, 16, 4]: 101.5
[512, 16, 4]: 101.5
[1024, 16, 4]: 101.5
[2048, 16, 4]: 101.5
[4096, 16, 4]: 101.5
[8192, 16, 4]: 101.5
[8, 3

In [140]:
for i in range(len(specification)):
    print(f"{specification[i]}: {cache_miss_per_diff_cache[i]}: {cycle_time_list[i]}: {miss_cost_list[i]}")

[256, 1, 4]: [543834, 9941, 30513, 3926638, 2478740, 447167, 3352240, 576650]: 0.6: 101.5
[512, 1, 4]: [527756, 8894, 24628, 3864093, 2460933, 428564, 3302713, 570041]: 0.6: 101.5
[1024, 1, 4]: [518257, 7683, 18529, 3828015, 2445561, 348747, 3263097, 567259]: 0.6: 101.5
[2048, 1, 4]: [513180, 6968, 15603, 3811240, 2440874, 307232, 3191233, 565957]: 0.6: 101.5
[4096, 1, 4]: [511395, 5097, 13449, 3802877, 2430627, 135262, 3161384, 564992]: 0.6: 101.5
[8192, 1, 4]: [510279, 5033, 10249, 3798687, 2093611, 24665, 3141962, 540571]: 0.6: 101.5
[16384, 1, 4]: [508894, 5033, 10154, 3780282, 2085763, 15036, 3132880, 538604]: 0.69: 101.5
[32768, 1, 4]: [504275, 5033, 10154, 3779851, 2084981, 15036, 3131905, 538504]: 0.85: 101.5
[65536, 1, 4]: [489317, 5033, 10154, 3763915, 2084981, 15036, 3131662, 538504]: 1.14: 101.5
[131072, 1, 4]: [438198, 5033, 10154, 3763798, 2084981, 15036, 3131662, 538504]: 1.35: 101.5
[128, 2, 4]: [534169, 10030, 21621, 3826921, 2454807, 343809, 3291018, 560066]: 0.6: 101

In [142]:
cpi_list = []

for i in range(len(specification)):
    tmp_list = []

    for j in range(8):
        # print(cache_miss_per_diff_cache[i][j])
        cpi= 1.2 + ((cache_miss_per_diff_cache[i][j]/number_of_ins[j]) * math.ceil(miss_cost_list[i]/cycle_time_list[i]))
        tmp_list.append(cpi)
    
    cpi_list.append(g_mean(tmp_list))

cpi_list

[4.699360963957171,
 4.54514317767521,
 4.337588576618608,
 4.2147627745846945,
 3.9034572142984145,
 3.6899201600799185,
 3.412298637173611,
 3.0711048621948236,
 2.6841927796575837,
 2.4754309796058664,
 4.496070291536222,
 4.344776615773177,
 4.233353304703971,
 4.1313441603772345,
 3.9206362954317044,
 3.4834574439646073,
 3.3169163793343115,
 2.843929720820689,
 2.578282465151751,
 2.3683235098673556,
 4.521933452495659,
 4.338855657752782,
 4.213295129428523,
 3.757994588718983,
 3.881979373766352,
 3.459699549235976,
 3.1889543239818496,
 2.8195939853245897,
 2.5254360793291766,
 2.329217979881386,
 4.531044967263453,
 4.068233333294213,
 3.8303536389974018,
 3.525918549026598,
 3.4105917377633577,
 3.1546148080274095,
 3.0014876268018407,
 2.653402469254859,
 2.3358985690313645,
 2.1490060192282923,
 4.5411146974920555,
 4.3696442002165865,
 3.0822127255396348,
 2.9215370787817485,
 2.7661563448197772,
 2.6876555316623687,
 2.6025572224088136,
 2.428940737107655,
 2.01794439561

In [144]:
instruction_per_program_gmean

array(152344.86346288)

In [148]:
cpu_time_list: list[float] = list()
IPP = float(instruction_per_program_gmean)

for i in range(len(specification)):
    cpu_time = IPP * cpi_list[i] * cycle_time_list[i]
    cpu_time_list.append(cpu_time)

In [153]:
for i in range(len(specification)):
    print(f"{specification[i]}: {cpu_time_list[i]}\n")




[256, 1, 4]: 429554.1026501178

[512, 1, 4]: 415457.5300933136

[1024, 1, 4]: 396485.6036778768

[2048, 1, 4]: 385258.47565353126

[4096, 1, 4]: 356802.99380730133

[8192, 1, 4]: 337284.22978579137

[16384, 1, 4]: 358693.85728261166

[32768, 1, 4]: 397686.9932745796

[65536, 1, 4]: 466172.20007849333

[131072, 1, 4]: 509110.9127097966

[128, 2, 4]: 410971.9288101694

[256, 2, 4]: 397142.640184018

[512, 2, 4]: 386957.7787171655

[1024, 2, 4]: 377633.43721851223

[2048, 2, 4]: 358373.2806691026

[4096, 2, 4]: 355560.18861530226

[8192, 2, 4]: 368880.0762370676

[16384, 2, 4]: 433258.0850164662

[32768, 2, 4]: 487057.2317516421

[65536, 2, 4]: 544810.9018374882

[64, 4, 4]: 413336.00064521976

[128, 4, 4]: 396601.4236593061

[256, 4, 4]: 385124.32273297396

[512, 4, 4]: 400757.8207588557

[1024, 4, 4]: 360753.7667739222

[2048, 4, 4]: 363676.5442611601

[4096, 4, 4]: 383798.44075034733

[8192, 4, 4]: 438141.673929345

[16384, 4, 4]: 504005.75124343485

[32768, 4, 4]: 560654.144290113

[3

In [156]:
with open("final_cpu_time_gmean.csv", "w") as fout:
    for i in range(len(specification)):
        csv_line = f"--dc={specification[i][0]}:{specification[i][1]}:{specification[i][2] * 4}, {cpu_time_list[i]}\n"
        fout.write(csv_line)

fout.close()

In [160]:
min_val = min(cpu_time_list)
cpu_time_list.index(min_val)

specification[75]

[4096, 1, 8]